In [1]:
import sys
sys.path.append("../")

In [2]:
import os
import json

import numpy as np
import mlflow

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
)
from tensorflow.keras.constraints import non_neg
from tensorflow.keras.callbacks import EarlyStopping

from src.utils import (
    mlflow_credential_path,
    get_metadata,
    mean_squared_error,
)

2022-11-07 13:41:38.508911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 13:41:38.508930: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## set environment variables

In [4]:
# read mlflow credentials
with open(mlflow_credential_path, "r") as f:
    credentials = json.load(f)

In [5]:
# set mlflow accounts
os.environ["MLFLOW_TRACKING_USERNAME"] = credentials["MLFLOW_TRACKING_USERNAME"]
os.environ["MLFLOW_TRACKING_PASSWORD"] = credentials["MLFLOW_TRACKING_PASSWORD"]

In [6]:
# set gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## load dataset

In [7]:
(X_train, y_train), (X_test, y_test) = get_metadata(train_valid_split=True)

## keras mlp model

In [8]:
def build_model(num_features):
    
    inputs = Input(shape=(num_features,))
    x = inputs
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation="relu")(x)
    outputs = Dense(1, kernel_constraint=non_neg())(x)

    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mse")
    
    return model

In [9]:
num_features = X_train.shape[1]

In [10]:
es_callback = EarlyStopping(patience=5)

In [11]:
model = build_model(num_features)

2022-11-07 13:41:49.406924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 13:41:49.407067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 13:41:49.407146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-07 13:41:49.407199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-07 13:41:49.407245: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [12]:
model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[es_callback]
)

2022/11/07 13:41:50 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current keras workflow


Epoch 1/100
140/140 [==============================] - 0s 1ms/step - loss: 1204.3831 - val_loss: 477.2576
Epoch 2/100
140/140 [==============================] - 0s 878us/step - loss: 482.4884 - val_loss: 452.6822
Epoch 3/100
140/140 [==============================] - 0s 1ms/step - loss: 458.0023 - val_loss: 436.4815
Epoch 4/100
140/140 [==============================] - 0s 1ms/step - loss: 444.7266 - val_loss: 430.1240
Epoch 5/100
140/140 [==============================] - 0s 2ms/step - loss: 437.6786 - val_loss: 426.6674
Epoch 6/100
140/140 [==============================] - 0s 1ms/step - loss: 436.9326 - val_loss: 425.4612
Epoch 7/100
140/140 [==============================] - 0s 1ms/step - loss: 434.6058 - val_loss: 424.9646
Epoch 8/100
140/140 [==============================] - 0s 2ms/step - loss: 432.3652 - val_loss: 428.3480
Epoch 9/100
140/140 [==============================] - 0s 1ms/step - loss: 433.5197 - val_loss: 425.0873
Epoch 10/100
140/140 [==============================

2022/11/07 13:41:54 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: API request to endpoint /api/2.0/mlflow/runs/update failed with error code 400 != 200. Response body: '"repo not associated with run"'


In [13]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [14]:
print(f"Train MSE: {mean_squared_error(y_train, pred_train)}")
print(f"Test MSE: {mean_squared_error(y_test, pred_test)}")

Train MSE: 20.51246884798047
Test MSE: 20.611175699329475
